In [1]:
# Data prepare: 2 classes [5,21] for quick check 
# VGG data prepare

import pandas as pd
import numpy as np
import csv
import json
import random
import matplotlib.pyplot as plt
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


train_file = '/root/data/S10.json'
with open(train_file) as data_file:
    train_anns=json.load(data_file)


img_5=[]
for i in train_anns['annotations']:
    if i['category_id'] == 5:
        img_5.append(i['image_id']+'.JPG')
img_5_1w = random.sample(img_5, 1300)

img_21=[]
for i in train_anns['annotations']:
    if i['category_id'] == 21:
        img_21.append(i['image_id']+'.JPG')
img_21_1w = random.sample(img_21,1300)

img_list = img_5_1w+img_21_1w

label_dict = {}
for i in train_anns['annotations']:
    i_new = i['image_id'] + '.JPG'
    if i_new in img_list:
        label_dict[i_new]=i['category_id']
        
df = pd.DataFrame.from_dict(label_dict,orient='index',columns=['label'])
df = df.reset_index().rename(columns={'index':'image_id'})
print(df.head())

df['label'].value_counts().plot.bar()
plt.show()

df.label.unique()

df.loc[df.label == 39, 'label']=21

df.label.unique()

df['label'].value_counts().plot.bar()
plt.show()

Using TensorFlow backend.


                                 image_id  label
0  S10/B03/B03_R1/S10_B03_R1_IMAG0979.JPG      5
1  S10/B03/B03_R1/S10_B03_R1_IMAG1043.JPG      5
2  S10/B03/B03_R1/S10_B03_R1_IMAG1111.JPG      5
3  S10/B03/B03_R1/S10_B03_R1_IMAG1115.JPG      5
4  S10/B03/B03_R2/S10_B03_R2_IMAG0004.JPG     21


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [2]:
df.label.unique()

array([ 5, 21])

In [3]:
df.shape

(2600, 2)

In [4]:
df.loc[df.label == 5, 'label']=0
df.loc[df.label == 21, 'label']=1

In [5]:
df.head()

,image_id,label
0,S10/B03/B03_R1/S10_B03_R1_IMAG0979.JPG,0
1,S10/B03/B03_R1/S10_B03_R1_IMAG1043.JPG,0
2,S10/B03/B03_R1/S10_B03_R1_IMAG1111.JPG,0
3,S10/B03/B03_R1/S10_B03_R1_IMAG1115.JPG,0
4,S10/B03/B03_R2/S10_B03_R2_IMAG0004.JPG,1


In [6]:
imglist = df['image_id'].tolist()
path = '/root/data/'

count = 0
nonelist = []
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path)
        count+=1
    except:
        nonelist.append(img_name)

len(nonelist)        

17

In [7]:
nonelist

['S10/E06/E06_R1/S10_E06_R1_IMAG0874.JPG',
 'S10/E06/E06_R1/S10_E06_R1_IMAG0875.JPG',
 'S10/E06/E06_R1/S10_E06_R1_IMAG0876.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0489.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0490.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0491.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0492.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0493.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0494.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0495.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0496.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0261.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0262.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0263.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0267.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0268.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0269.JPG']

In [8]:
df1=df[~df.image_id.isin(nonelist)]
len(df1)

2583

In [9]:
train_df, validate_df = train_test_split(df1, test_size=0.20, random_state=523)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [10]:
len(train_df)

2066

In [11]:
len(validate_df)

517

In [12]:
imglist = train_df['image_id'].tolist()

In [13]:
X_train = np.empty((len(imglist),227,227,3))

In [14]:
Y_train = np.empty((len(imglist),1))

In [15]:
X_train.shape

(2066, 227, 227, 3)

In [16]:
Y_train.shape

(2066, 1)

In [17]:
path = '/root/data/'

In [18]:
imglist[:2]

['S10/P13/P13_R3/S10_P13_R3_IMAG0422.JPG',
 'S10/M09/M09_R3/S10_M09_R3_IMAG0569.JPG']

In [19]:
count=0
nonelist=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        X_train[count]=img
        count+=1
    except:
        nonelist.append(img_path)

In [20]:
len(nonelist)

0

In [21]:
X_train[:1]

array([[[[150., 178., 226.],
         [143., 180., 225.],
         [149., 175., 224.],
         ...,
         [  1.,   7.,   5.],
         [  1.,   7.,   5.],
         [  1.,   7.,   5.]],

        [[143., 180., 225.],
         [144., 183., 226.],
         [146., 177., 224.],
         ...,
         [  1.,   7.,   5.],
         [  1.,   7.,   5.],
         [  1.,   7.,   5.]],

        [[145., 176., 223.],
         [145., 178., 223.],
         [148., 176., 223.],
         ...,
         [  1.,   7.,   5.],
         [  1.,   7.,   5.],
         [  1.,   7.,   7.]],

        ...,

        [[220., 220., 220.],
         [220., 220., 220.],
         [221., 221., 221.],
         ...,
         [224., 224., 224.],
         [224., 224., 224.],
         [221., 221., 219.]],

        [[220., 220., 220.],
         [220., 220., 220.],
         [220., 220., 220.],
         ...,
         [218., 218., 218.],
         [220., 220., 220.],
         [218., 218., 218.]],

        [[221., 219., 222.],
       

In [22]:
label = train_df['label']
Y_train = [i for i in label]

In [23]:
type(Y_train)

list

In [24]:
type(X_train)

numpy.ndarray

In [25]:
d=np.array(Y_train)

In [27]:
Y_train=d.reshape(2066,1)

In [28]:
Y_train.shape

(2066, 1)

In [29]:
train_df[:2]

,image_id,label
0,S10/P13/P13_R3/S10_P13_R3_IMAG0422.JPG,0
1,S10/M09/M09_R3/S10_M09_R3_IMAG0569.JPG,1


In [30]:
Y_train[:2]

array([[0],
       [1]])

In [31]:
imglist = validate_df['image_id'].tolist()
X_validate = np.empty((len(imglist),227,227,3))
X_validate.shape

(517, 227, 227, 3)

In [32]:
count=0
nonelist_validate=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        X_validate[count]=img
        count+=1
    except:
        nonelist_validate.append(img_path)

In [33]:
X_validate.shape

(517, 227, 227, 3)

In [34]:
label = validate_df['label']
Y_validate = [i for i in label]
Y_validate = np.array(Y_validate)
Y_validate = Y_validate.reshape(len(Y_validate),1)
Y_validate.shape

(517, 1)

In [35]:
validate_df[:2]

,image_id,label
0,S10/P13/P13_R3/S10_P13_R3_IMAG0521.JPG,0
1,S10/M09/M09_R3/S10_M09_R3_IMAG0621.JPG,1


In [36]:
Y_validate[:2]

array([[0],
       [1]])

modeling

In [42]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.datasets import cifar10
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import numpy as np


classes_num = 2
batch_size = 32
epochs_num = 100

In [43]:
# -- coding: utf-8 --
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.datasets import cifar10
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import numpy as np


classes_num = 2
batch_size = 32
epochs_num = 100

def quality_classify_model():
    model = Sequential()
    model.add(Flatten(input_shape=(4,4,512)))# 4*4*512
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_num, activation='softmax'))  # ???

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def train():
    # ????
    x_train = X_train
    y_train = Y_train
    x_test = X_validate
    y_test = Y_validate

    # ???????
    y_train = keras.utils.to_categorical(y_train, classes_num)
    y_test = keras.utils.to_categorical(y_test, classes_num)
    # ??????
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    datagan = ImageDataGenerator(rescale=1./255)

    # ??????????
    conv_base = VGG16(include_top=False, weights='imagenet')

    # ??????????????????
    sample_count = len(y_train)
    train_features = np.zeros(shape=(sample_count, 4, 4, 512))
    train_labels = np.zeros(shape=(sample_count, classes_num))
    train_generator = datagan.flow(x_train, y_train, batch_size=batch_size)
    i = 0
    for inputs_batch, labels_batch in train_generator:
        features_batch = conv_base.predict(inputs_batch)
        train_features[i * batch_size : (i + 1) * batch_size] = features_batch
        train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # train_features = np.reshape(train_features, (sample_count, 4*4*512))

    # ??????????????????
    sample_count = len(y_test)
    test_generator = datagan.flow(x_test, y_test, batch_size=batch_size)
    test_features = np.zeros(shape=(sample_count, 4, 4, 512))
    test_labels = np.zeros(shape=(sample_count, classes_num))
    i = 0
    for inputs_batch, labels_batch in test_generator:
        features_batch = conv_base.predict(inputs_batch)
        test_features[i * batch_size : (i + 1) * batch_size] = features_batch
        test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    test_features = np.reshape(test_features, (sample_count, 4*4*512))

    model = quality_classify_model()

    # hist = model.fit_generator(train_datagan.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch = 8000, epochs = epochs_num, validation_data=(x_test,y_test), shuffle=True)
    hist = model.fit(train_features, train_labels, batch_size=batch_size, epochs=epochs_num, validation_data=(test_features, test_labels))

    model.save('./extract_features/cifar10_model.hdf5') 
    model.save_weights('./extract_features/cifar10_model_weight.hdf5')

    hist_dict = hist.history
    print("train acc:")
    print(hist_dict['acc'])
    print("validation acc:")
    print(hist_dict['val_acc'])

    train_acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    train_loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    # ??
    epochs = range(1, len(train_acc)+1)
    plt.plot(epochs, train_acc, 'bo', label = 'Training acc')
    plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig("accuracy.png")
    plt.figure() # ?????
    plt.plot(epochs, train_loss, 'bo', label = 'Training loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig("loss.png")

In [44]:
train()

ValueError: could not broadcast input array from shape (32,7,7,512) into shape (32,4,4,512)

try another vgg transfer learning code

#### sample-2 model

In [ ]:
from keras.layers import Flatten,Dense,Dropout,Input
from keras.applications import VGG16
# from load_data import load_data_split
from keras.optimizers import SGD
from keras.models import Model 
from keras.utils import np_utils
# import
print('[INFO] loading dataset......')
(x_train,y_train)=(X_train, Y_train)
(x_valid,y_valid)=(X_validate, Y_validate)

y_train=np_utils.to_categorical(y_train,2)
y_valid=np_utils.to_categorical(y_valid,2)
print('[INFO] initializing model......')
base_model=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(227,227,3)))

#微调
head_model=base_model.output
head_model=Flatten(name="flatten")(head_model)
head_model = Dense(512, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model=Dense(32,activation='relu')(head_model)
head_model = Dense(2, activation="softmax")(head_model)
model=Model(base_model.input,head_model)

#冻结前面的5个卷积组，只训练自定义的全连接层
for layer in base_model.layers:
    layer.trainable=False
print('[INFO] compiling model')
sgd=SGD(lr=0.0001,momentum=0.9)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=sgd)
print('[INFO] training model')
model.fit(x_train, y_train, batch_size=32, epochs=40, validation_data=(x_valid,y_valid))
print('[INFO] saving model and weights')
#保存模型（不含权重）
model_json=model.to_json()
open('model_architecture.json','w').write(model_json)
#保存权重
model.save_weights('transfer_learning_weights.h5', overwrite=True)

[INFO] loading dataset......
[INFO] initializing model......
[INFO] compiling model
[INFO] training model
Train on 2066 samples, validate on 517 samples
Epoch 1/40
2066/2066 [==============================] - 566s 274ms/step - loss: 1.5770 - acc: 0.7657 - val_loss: 0.3707 - val_acc: 0.8337
Epoch 2/40
2066/2066 [==============================] - 563s 273ms/step - loss: 0.4361 - acc: 0.8746 - val_loss: 0.1871 - val_acc: 0.9381
Epoch 3/40
2066/2066 [==============================] - 563s 273ms/step - loss: 0.3267 - acc: 0.8925 - val_loss: 0.2366 - val_acc: 0.9188
Epoch 4/40
2066/2066 [==============================] - 564s 273ms/step - loss: 0.2542 - acc: 0.9119 - val_loss: 0.2039 - val_acc: 0.9284
Epoch 5/40
2066/2066 [==============================] - 564s 273ms/step - loss: 0.2049 - acc: 0.9303 - val_loss: 0.1641 - val_acc: 0.9497
Epoch 6/40
2066/2066 [==============================] - 564s 273ms/step - loss: 0.1896 - acc: 0.9371 - val_loss: 0.1487 - val_acc: 0.9536
Epoch 7/40
2066/206